# RAMP Sarting kit on fire detection

## Table of Contents

- [Introduction](#introduction)
- [The Dataset](#the-dataset)
- [Preprocessing](#Preprocessing)
- [Requirements](#Requirements)
- [Data exploration](#data-exploration)
- [Base Model](#base-model)

## Introduction

The objective of this challenge is to predict the occurrence of wildfires based on meteorological data and information about the population in French municipalities.

## The Dataset

We have merged several datasets to gather diverse information regarding the occurrence of wildfires. The meteorological data is sourced from Météo France and has been merged with information on wildfire incidents in French municipalities. This merging process has resulted in a comprehensive database that allows us to analyze the occurrence of wildfires based on specific characteristics.

For this challenge, the data underwent preprocessing and was subsequently divided to maintain a private test set. This test set will be used to evaluate the models on our servers.

Please review and provide any additional details or clarifications if needed.


## Requirements

In [ ]:
import geoviews as gv
import numpy as np
import pandas as pd
import geoviews.feature as gf
import xarray as xr
from cartopy import crs
from geoviews import dim
import geopandas as gpd
import matplotlib.pyplot as plt

## Preprocessing

Our database is composed of several merged datasets. For the simplicity of the challenge, we provide you with the already merged data, and we have dropped non-essential data in advance. Our dataset now contains only the most relevant features.

In [3]:
train = pd.read_csv('data/train/X_train.csv')
test = pd.read_csv('data/test/public/X_test_public.csv')


In [7]:
train = pd.get_dummies(train, columns = ['Date',
                                         'communes (name)',
                                         'communes (code)',
                                         'department (name)',
                                         'region (name)', 
                                         'Type Touristique',
                                         'Libellé typologie'
                                        ])

test = pd.get_dummies(test, columns = ['Date',
                                         'communes (name)',
                                         'communes (code)',
                                         'department (name)',
                                         'region (name)', 
                                         'Type Touristique',
                                         'Libellé typologie'
                                        ])

## Data exploration


In [8]:
train

,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Humidité,Précipitations dans les 24 dernières heures,Latitude,Longitude,Altitude,Température minimale du sol sur 12 heures (en °C),region (code),...,region (name)_Saint-Barthélemy,region (name)_Saint-Pierre-et-Miquelon,region (name)_Terres australes et antarctiques françaises,region (name)_Île-de-France,Type Touristique_Commune touristique,Type Touristique_Station classée de tourisme,Libellé typologie_Communes de densité intermédiaire,Libellé typologie_Communes densément peuplées,Libellé typologie_Communes peu denses,Libellé typologie_Communes très peu denses
0,130.0,2.9,286.75,90.0,-0.1,47.150000,-1.608833,26,5.5,52.0,...,False,False,False,False,False,False,True,False,False,False
1,270.0,3.3,286.35,75.0,0.2,47.059167,2.359833,161,2.7,24.0,...,False,False,False,False,True,False,False,True,False,False
2,170.0,1.9,284.05,88.0,-0.1,50.136000,1.834000,69,7.6,32.0,...,False,False,False,False,False,False,True,False,False,False
3,230.0,2.2,285.45,88.0,0.0,49.209667,4.155333,95,10.9,44.0,...,False,False,False,False,False,False,False,False,True,False
4,240.0,6.7,286.55,91.0,-0.1,50.570000,3.097500,47,9.2,32.0,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17980,210.0,8.3,280.55,77.0,3.8,-37.795167,77.569167,27,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
17981,260.0,4.0,291.95,74.0,4.2,46.593833,0.314333,123,17.5,75.0,...,False,False,False,False,True,False,False,True,False,False
17982,210.0,4.1,289.85,80.0,5.6,48.445500,0.110167,143,17.5,28.0,...,False,False,False,False,False,False,True,False,False,False
17983,200.0,2.3,302.55,58.0,0.0,41.918000,8.792667,5,25.9,94.0,...,False,False,False,False,False,True,True,False,False,False


In [9]:
train.columns


Index(['Direction du vent moyen 10 mn', 'Vitesse du vent moyen 10 mn',
       'Température', 'Humidité',
       'Précipitations dans les 24 dernières heures', 'Latitude', 'Longitude',
       'Altitude', 'Température minimale du sol sur 12 heures (en °C)',
       'region (code)',
       ...
       'region (name)_Saint-Barthélemy',
       'region (name)_Saint-Pierre-et-Miquelon',
       'region (name)_Terres australes et antarctiques françaises',
       'region (name)_Île-de-France', 'Type Touristique_Commune touristique',
       'Type Touristique_Station classée de tourisme',
       'Libellé typologie_Communes de densité intermédiaire',
       'Libellé typologie_Communes densément peuplées',
       'Libellé typologie_Communes peu denses',
       'Libellé typologie_Communes très peu denses'],
      dtype='object', length=358)

In [10]:
train.dtypes

Direction du vent moyen 10 mn                          float64
Vitesse du vent moyen 10 mn                            float64
Température                                            float64
Humidité                                               float64
Précipitations dans les 24 dernières heures            float64
                                                        ...   
Type Touristique_Station classée de tourisme              bool
Libellé typologie_Communes de densité intermédiaire       bool
Libellé typologie_Communes densément peuplées             bool
Libellé typologie_Communes peu denses                     bool
Libellé typologie_Communes très peu denses                bool
Length: 358, dtype: object

## Base Model